In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.statespace.sarimax import SARIMAX
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# Load and prepare the data
data = pd.read_csv("demand_inventory.csv")
data['Date'] = pd.to_datetime(data['Date'], format='%d-%m-%Y')

In [3]:
# List of unique products
products = data['Product'].unique()

In [4]:
# Parameters
future_steps = 10
lead_time = 1
service_level = 0.95
holding_cost = 0.1
stockout_cost = 10

results = []
demand_forecasts = []


In [5]:
future_steps = 30  # So user can select any start date within next 60 days

demand_forecasts = []
results = []

for product in products:
    print(f"\n=== Forecasting for {product} ===")
    product_data = data[data['Product'] == product].copy()
    product_data = product_data.sort_values('Date')
    time_series = product_data.set_index('Date')['Demand']

    # Fit SARIMAX model
    order = (1, 1, 1)
    seasonal_order = (1, 1, 1, 2)
    model = SARIMAX(time_series, order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)

    # Forecast future demand
    predictions = model_fit.predict(len(time_series), len(time_series) + future_steps - 1).astype(int)
    future_dates = pd.date_range(start=time_series.index[-1] + pd.DateOffset(days=1), periods=future_steps, freq='D')
    forecasted_demand = pd.Series(predictions, index=future_dates)
    demand_forecasts.append((product, forecasted_demand))

    # Inventory optimization calculations
    initial_inventory = product_data['Inventory'].iloc[-1]
    z = np.abs(np.percentile(forecasted_demand, 100 * (1 - service_level)))
    order_quantity = np.ceil(forecasted_demand.mean() + z).astype(int)
    reorder_point = forecasted_demand.mean() * lead_time + z
    safety_stock = reorder_point - forecasted_demand.mean() * lead_time
    total_holding_cost = holding_cost * (initial_inventory + 0.5 * order_quantity)
    total_stockout_cost = stockout_cost * np.maximum(0, forecasted_demand.mean() * lead_time - initial_inventory)
    total_cost = total_holding_cost + total_stockout_cost

    print(f"Forecasted Demand: {product} \n", forecasted_demand)
    print("Optimal Order Quantity:", order_quantity)
    print("Reorder Point:", reorder_point)
    print("Safety Stock:", safety_stock)
    print("Total Cost:", total_cost)

    results.append({
        "Product": product,
        "Order Quantity": order_quantity,
        "Reorder Point": reorder_point,
        "Safety Stock": safety_stock,
        "Total Cost": total_cost
    })

results_df = pd.DataFrame(results)
print("\n=== Summary ===")
print(results_df)



=== Forecasting for Smartphone ===


c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Forecasted Demand: Smartphone 
 2025-03-02    96
2025-03-03    96
2025-03-04    95
2025-03-05    95
2025-03-06    95
2025-03-07    95
2025-03-08    94
2025-03-09    94
2025-03-10    94
2025-03-11    94
2025-03-12    93
2025-03-13    93
2025-03-14    93
2025-03-15    93
2025-03-16    93
2025-03-17    93
2025-03-18    92
2025-03-19    92
2025-03-20    92
2025-03-21    92
2025-03-22    91
2025-03-23    91
2025-03-24    91
2025-03-25    91
2025-03-26    91
2025-03-27    90
2025-03-28    90
2025-03-29    90
2025-03-30    90
2025-03-31    90
Freq: D, Name: predicted_mean, dtype: int32
Optimal Order Quantity: 183
Reorder Point: 182.63333333333333
Safety Stock: 89.99999999999999
Total Cost: 470.45000000000005

=== Forecasting for Laptop ===


c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Forecasted Demand: Laptop 
 2025-03-02    78
2025-03-03    85
2025-03-04    79
2025-03-05    84
2025-03-06    79
2025-03-07    84
2025-03-08    79
2025-03-09    83
2025-03-10    79
2025-03-11    83
2025-03-12    79
2025-03-13    83
2025-03-14    78
2025-03-15    82
2025-03-16    78
2025-03-17    82
2025-03-18    78
2025-03-19    82
2025-03-20    78
2025-03-21    82
2025-03-22    77
2025-03-23    81
2025-03-24    77
2025-03-25    81
2025-03-26    77
2025-03-27    81
2025-03-28    76
2025-03-29    80
2025-03-30    76
2025-03-31    80
Freq: D, Name: predicted_mean, dtype: int32
Optimal Order Quantity: 157
Reorder Point: 156.48333333333335
Safety Stock: 76.45000000000002
Total Cost: 552.95

=== Forecasting for Tablet ===


...

Forecasted Demand: Gaming Console 
 2025-03-02    143
2025-03-03    142
2025-03-04    142
2025-03-05    140
2025-03-06    142
2025-03-07    140
2025-03-08    142
2025-03-09    140
2025-03-10    142
2025-03-11    140
2025-03-12    142
2025-03-13    140
2025-03-14    142
2025-03-15    140
2025-03-16    142
2025-03-17    140
2025-03-18    142
2025-03-19    140
2025-03-20    142
2025-03-21    140
2025-03-22    142
2025-03-23    140
2025-03-24    142
2025-03-25    140
2025-03-26    142
2025-03-27    140
2025-03-28    142
2025-03-29    140
2025-03-30    142
2025-03-31    140
Freq: D, Name: predicted_mean, dtype: int32
Optimal Order Quantity: 282
Reorder Point: 281.1
Safety Stock: 140.00000000000003
Total Cost: 635.1

=== Forecasting for Digital Camera ===


c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Forecasted Demand: Digital Camera 
 2025-03-02    108
2025-03-03    105
2025-03-04    109
2025-03-05    106
2025-03-06    109
2025-03-07    106
2025-03-08    109
2025-03-09    106
2025-03-10    110
2025-03-11    106
2025-03-12    110
2025-03-13    106
2025-03-14    110
2025-03-15    106
2025-03-16    110
2025-03-17    106
2025-03-18    110
2025-03-19    106
2025-03-20    110
2025-03-21    106
2025-03-22    110
2025-03-23    107
2025-03-24    110
2025-03-25    107
2025-03-26    110
2025-03-27    107
2025-03-28    110
2025-03-29    107
2025-03-30    111
2025-03-31    107
Freq: D, Name: predicted_mean, dtype: int32
Optimal Order Quantity: 214
Reorder Point: 214.0
Safety Stock: 106.0
Total Cost: 593.1

=== Forecasting for E-Reader ===


c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)
c:\Users\KISHORE\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  self._init_dates(dates, freq)


Forecasted Demand: E-Reader 
 2025-03-02    91
2025-03-03    83
2025-03-04    92
2025-03-05    83
2025-03-06    92
2025-03-07    83
2025-03-08    92
2025-03-09    83
2025-03-10    92
2025-03-11    83
2025-03-12    92
2025-03-13    84
2025-03-14    92
2025-03-15    84
2025-03-16    92
2025-03-17    84
2025-03-18    92
2025-03-19    84
2025-03-20    92
2025-03-21    84
2025-03-22    93
2025-03-23    84
2025-03-24    93
2025-03-25    84
2025-03-26    93
2025-03-27    84
2025-03-28    93
2025-03-29    84
2025-03-30    93
2025-03-31    84
Freq: D, Name: predicted_mean, dtype: int32
Optimal Order Quantity: 171
Reorder Point: 170.96666666666667
Safety Stock: 83.0
Total Cost: 470.95000000000005

=== Summary ===
             Product  Order Quantity  Reorder Point  Safety Stock  Total Cost
0         Smartphone             183     182.633333         90.00      470.45
1             Laptop             157     156.483333         76.45      552.95
2             Tablet             198     197.733333  

In [6]:
import plotly.graph_objects as go

# --- Combine all forecasts into one figure ---
fig = go.Figure()

for product, series in demand_forecasts:
    series = series.sort_index()  # Ensure the forecast is in chronological order
    fig.add_trace(
        go.Scatter(
            x=series.index,
            y=series.values,
            mode='lines+markers',
            name=product
        )
    )

fig.update_layout(
    title="Forecasted Demand for All Products",
    xaxis_title="Date",
    yaxis_title="Forecasted Demand",
    legend_title="Product",
    width=1000,
    height=600
)

# Save as HTML
fig.write_html("forecasted_demand_all_products.html")
print("📈 Combined forecast plot saved to 'forecasted_demand_all_products.html'")


📈 Combined forecast plot saved to 'forecasted_demand_all_products.html'


In [7]:
# --- Save results to CSVs ---
results_df.to_csv("inventory_optimization_summary.csv", index=False)

forecast_rows = []
for product, series in demand_forecasts:
    for date, value in series.items():
        forecast_rows.append({
            'Product': product,
            'Date': date,
            'Forecasted Demand': value
        })
forecast_df = pd.DataFrame(forecast_rows)
forecast_df.to_csv("forecasted_demand_all_products.csv", index=False)

# --- Generate Forecast Subplots ---
cols =1
rows = int(np.ceil(len(demand_forecasts) / cols))
subplot_fig = make_subplots(rows=rows, cols=cols, subplot_titles=[prod for prod, _ in demand_forecasts])

row = 1
col = 1
for product, series in demand_forecasts:
    subplot_fig.add_trace(
        go.Scatter(x=series.index, y=series.values, mode='lines+markers', name=product, showlegend=False),
        row=row, col=col
    )
    col += 1
    if col > cols:
        row += 1
        col = 1

subplot_fig.update_layout(height=400 * rows, width=1000, title_text="Forecasted Demand per Product (Subplots)")
subplot_fig.write_html("forecasted_demand_subplots.html")
print("📈 Forecast plot saved to 'forecasted_demand_subplots.html'")


📈 Forecast plot saved to 'forecasted_demand_subplots.html'
